### Input/output options for WHISP

#### Imports/setup

In [1]:
import ee
import pandas as pd
import geopandas as gpd
import json
import geemap
import time
import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from modules.file_to_ceo import get_ceo_url, whisp_stats_shapefile_and_ceo_url
from parameters.config_output_naming import geo_id_column, out_file_wide, out_shapefile
from modules.agstack_to_gee import start_agstack_session, get_agstack_token, geometry_to_geo_id, feature_to_geo_id,geo_id_or_ids_to_feature_collection, register_feature_and_set_geo_id, register_fc_and_set_geo_id, shapefile_to_ee_feature_collection 
from parameters.config_asr_url_info import asset_registry_base,user_registry_base
from parameters.config_asr_credentials import email,password

def whisp_alt_stats_as_df (roi): 
    %store roi
    %run alt_whisp_stats.ipynb   
    return df_out

#set up asset registry
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)


Earth Engine has been initialized with the specified credentials.


#### Input: list of Geo ids

i.e. Geo ids from fields registered in the asset registry: https://asset-registry.agstack.org/

In [2]:
GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids

#make into a feature collection
roi = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

#Whisp stats as a dataframe
df = whisp_alt_stats_as_df(roi)

df

Stored 'roi' (FeatureCollection)
...ee stats processing time: 2.3085296154022217 seconds
...ee to df processing time: 5.285902261734009 seconds
...table tidying time: 0.011044025421142578 seconds.
 Processing finished


,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,1.9,GHA,19,0,0,100,20,85,100,18,-,0,0,0,0,-
2,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,16.6,GHA,100,87,95,100,100,89,100,5,True,0,0,0,0,True
3,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,31.2,IDN,0,0,0,6,1,71,93,0,-,100,99,82,0,-
4,8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53...,20.9,IDN,28,0,31,96,54,96,100,44,-,0,0,51,0,-
5,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,8.3,CIV,5,0,0,95,61,21,77,4,-,0,0,0,0,True
6,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...,3.6,CIV,41,0,0,100,46,100,98,6,-,0,0,0,0,-


#### Output: stats to CSV file

In [6]:
#export output csv
df.to_csv(path_or_buf=out_file_wide, header=True)

print ("output csv: ", out_file_wide)

output csv:  /home/sepal-user/whisp/whisp_output_table.csv


#### Output: Collect Earth Online (CEO) link

Temporary login details available for demo purposes here: modules/config_ceo.py

In [ ]:
ceo_url = whisp_stats_shapefile_and_ceo_url(roi, df, plot_id_column, out_shapefile)

print('CEO URL:', ceo_url) 

### Other input options:

#### Input: shapefile

In [ ]:
roi = shapefile_to_ee_feature_collection(out_shapefile)

df= whisp_alt_stats_as_df(roi)
df

#### Input: GEE asset (feature collection) 

In [4]:
example_plot_feature_col_asset = "projects/fdap-remi/assets/seg_civ_scoopac"
fc = ee.FeatureCollection(example_plot_feature_col_asset).limit(500)

#run for feature collection (NB needs speeding up..get geo ids then use map over property setting)
roi = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=False
)

df = whisp_alt_stats_as_df(roi)
df

Field boundary registered successfully!
Response: 8acdb733a730afe4719221753ef93549cf6d129237d928058ad48f1e8ea18878
Field boundary registered successfully!
Response: 19f868b55bcf63c714253035794a2f9bd89c05448c430dc701dacb88a9a94f22
Field boundary registered successfully!
Response: 0a240c05046e3703e543443af9dbd6da048671f2ad290d300d507698787e5143
Field boundary registered successfully!
Response: 5a3bafc9f7aa5f0efedee60d24a5a22c3182d61d1494740c8a9e2b01d2b1b55f
Field boundary registered successfully!
Response: 5fcf6f664b48bba50e1f1dd718160e4942d83013dd387c548f99a3c8df84d4aa
Field boundary registered successfully!
Response: 0ebe32f387b6b281ad0dc96e495b3071f26f523b5571e8222a44eed4b6038c4b
Field boundary registered successfully!
Response: 16c83494e08094be7200ada97cf966d0aca3b8f412677a2d599996bfcf3e5314
Field boundary registered successfully!
Response: d82d02578d83898e9d8d5a622ad93a29223c6a929e03d97b5e681cd52fcfac10
Field boundary registered successfully!
Response: 3f51472d82dceec45b98cf6a5da623

,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,11ac3c0b57353f173747a0275cee3a00187e0a73458f3a...,0.0,CIV,100,0,0,100,100,100,100,0,-,0,0,0,0,-
2,d091a16acf45e2a855ae6be628a1d3d02724df7bc36ff3...,0.0,CIV,100,0,0,100,100,0,100,0,-,0,0,0,100,-
3,5086df340ad747fd4684d7fd406caa3536f576add47c35...,0.0,CIV,0,0,0,100,0,100,0,0,-,0,0,0,100,-
4,6057cc646a1f0355ceb7513a828981be4e7384b87ecabb...,0.0,CIV,98,0,0,100,100,98,100,0,-,0,0,0,0,-
5,9c0a96fe0c7449b8009a618c4720002a88fed9db9dc498...,0.0,CIV,100,0,100,100,100,100,100,0,-,0,0,0,0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,d,0.9,CIV,85,0,0,100,96,96,72,0,-,15,0,0,0,-
497,1,0.9,CIV,84,0,0,100,97,94,77,30,-,0,0,0,0,True
498,5,0.9,CIV,100,0,19,100,100,100,100,78,-,0,0,0,0,True


#### Input: polygon feature collection

In [ ]:
geom_1 =ee.Geometry.Polygon(
        [[[102.19867776108482, 3.0835809888660344],
          [102.1988172359536, 3.082670357801012],
          [102.20001886559227, 3.082078035157836],
          [102.2013385124276, 3.081745922272887],
          [102.2014355657636, 3.0825815609469625],
          [102.20011591892828, 3.0836528916202233]]],None, False);

geom_2 = ee.Geometry.Polygon(
        [[[102.18777620396249, 3.079459767638651],
          [102.18831264576546, 3.0785384196951506],
          [102.1895786484205, 3.0792026473624907],
          [102.18899929127328, 3.08005971470826]]],None, False);

feat_1 = ee.Feature(geom_1)
feat_2 = ee.Feature(geom_2)

fc = ee.FeatureCollection([feat_1,feat_2])

#add geo ids to feature collection
roi = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=False
)

#get stats
df = whisp_alt_stats_as_df(roi)
df

#### Input: point feature collection

In [ ]:
#get test points from polygon feature collection (NB functionality for registering points in asset registry coming soon)

# Compute centroids of each polygon
def getCentroid(feature):
    keepProperties = [geo_id_column];
    # Get the centroid of the feature's geometry.
    centroid = feature.geometry().centroid(1);
    #Return a new Feature, copying properties from the old Feature.
    return ee.Feature(centroid).copyProperties(feature, keepProperties);

roi = ee.FeatureCollection(fc.map(getCentroid))


whisp_alt_stats_as_df(roi)

#### Input: drawn feature

In [ ]:
zoom_level =15

Map = geemap.Map(center=[6,-4], zoom=15)

# Map.add_basemap("Esri.WorldImagery")# optional basemaps from geemap

sc = Sidecar(title='draw polygon')

with sc:display(Map)
print("Draw shape on map before running next cell")

In [ ]:
#NB User input needed: draw boundary (under 400ha) on the map before running this cell

drawn_feature = (Map.draw_last_feature)

#register geo id and add to feature
feature_w_geo_id = register_feature_and_set_geo_id(
    drawn_feature,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True)

roi = feature_w_geo_id

df = whisp_alt_stats_as_df(ee.FeatureCollection(roi))

df